# Imports

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops

from datasets import load_dataset
from random import randrange

import torch
from transformers import AutoTokenizer,AutoModelForCausalLM,AutoModelForSeq2SeqLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from trl import SFTTrainer #supervised finetuning
from huggingface_hub import login

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.4 MB/s eta 0:00:00


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from transformers import AutoTokenizer, GPTNeoXForCausalLM, GPTNeoXConfig


model_name = "EleutherAI/polyglot-ko-1.3b"

#Loading the Pre-trained Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    use_cache=False,
    device_map="auto")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00001-of-00003.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/748M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
print(model)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 2048)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
  

# Apply LoRA

In [5]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

#create the trainer
lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" #predicting the next token in the sequence
)

base_model = prepare_model_for_kbit_training(model)
model = get_peft_model(base_model, lora_config)



In [6]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 2048)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-23): 24 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=6144, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (defaul

# Data

In [7]:
import json

# Load data from the original JSON file
with open('/content/전문가_라벨링_데이터_고등학교.json', 'r', encoding='utf-8') as file:
    original_data = json.load(file)

# Process the data to extract summaries and combine job labels with expert comments
processed_data = []
for key, value in original_data.items():
    summaries = [summary["summary"] for summary in value["counselling_summaries"]]
    job_label = value["job_label"]
    expert_comment = value["expert_comment"]["ko"]

    # Creating a dictionary for each student
    student_data = {
        "Summary": " ".join(summaries),
        "Response": f"{job_label}: {expert_comment}"
    }
    processed_data.append(student_data)

# Save processed data to a new JSON file
with open('/content/processed_data.json', 'w', encoding='utf-8') as outfile:
    json.dump(processed_data, outfile, ensure_ascii=False, indent=4)


In [8]:
with open('/content/processed_data.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)

dataset[:1]

[{'Summary': '학교 성적이 좋은 편이며, 자격증도 많이 취득한 상황이다. 취업을 하는 것이 더 좋다는 얘기를 들었지만, 미용 쪽으로의 대학 진학을 준비하고 있다. 가고 싶은 대학이 정해져 있으며, 학교 미용 선생님을 꿈꾸고 있다. 그러나 미용 선생님이 되는 것이 어려운 길이라고 생각하고 있으며, 자신의 약한 체력에 대해서도 고민하고 있다. 만약 취업한다면 회계사무원 쪽으로 할 예정이다. 지난 주에 원하는 곳은 아니지만 안정적인 곳에 자기소개서를 제출했다. 대기업이나 중견기업과 같은 안정적인 곳으로 취업하고 싶으며, 화장품 쪽 회사도 고려하고 있다. 안정성과 소속감을 중요한 가치로 생각하며, 취업을 하지 않을 경우 부모님께 학비를 빌려 대학진학을 고려하고 있다. 주변에서 다정하고 화를 잘 안내는 것이 장점이라고 했으며, 단점은 우유부단한 것이라고 했다. 유튜브를 통해 직업에 대한 정보를 얻고 있다.',
  'Response': '사무계열:   고등학교 3학년인 이 여학생은 미용 교사를 목표로 진학과 취업을 동시에 준비하는 진로 탐색 단계에 있습니다.\n  이 학생은 꾸미는 것을 좋아하고 학교 공부를 잘하며 다양한 자격증을 취득하고 있습니다. 운동능력은 좋으나 체력이 좋지 않다고 했으며, 학교에서 취업을 권하지만, 미용 대학 진학을 희망하고 있다고 했습니다. 원하는 직장이 나타나면 먼저 회계사무원으로 취업하고 공부를 하겠다고 했습니다. 안정적인 대기업이나 중견 기업 또는 화장품 회사 취업을 원한다고 했습니다. 직업 심리 검사에서 예술형/사회형/관습형이 높게 나왔고, 적성에서 시각예술이 가장 높았으며, 언어 능력과 수리 능력도 높게 나왔습니다. MBTI는 할 때마다 다르며 INFP(조용하고 느긋하다. 꿈이 많고, 호기심 많은, 창의적)와 ENFP(열정적이고 사교적이고 순수하다, 창의적, 활동적)인데 INFP에 좀 더 가까운 것 같다고 했습니다. 주변에서 다정하고 화를 잘 안 낸다고 했으며, 본인은 이성적이고 남의 감정을 중요하게 생각한다고 했습니다. 선택

In [14]:
from transformers import  TrainingArguments

trainingArgs = TrainingArguments(
    output_dir = '/output',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit", # from the QLoRA paper
    logging_steps=1,
    save_strategy="epoch",
    learning_rate=2e-3,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=True
)

In [15]:
def prompt_instruction_format(sample):
    instruction = "학생의 진로 고민을 토대로 진로 상담사처럼 조언을 해주세요."
    formatted_prompt = instruction + "\n 학생 고민: " + sample['Summary'] + "\n 상담사 답변:" + sample['Response']
    return formatted_prompt


In [25]:
max_seq_length = 2048

params_to_exclude_from_gradients = []
# Set requires_grad to False for the specified parameters
for param in params_to_exclude_from_gradients:
    param.requires_grad = False

trainer = SFTTrainer(
    model= model,
    train_dataset=dataset[:50],
    peft_config = lora_config,
    max_seq_length=max_seq_length,
    tokenizer = tokenizer,
    packing=True,
    formatting_func = prompt_instruction_format,
    args= trainingArgs,
)

Generating train split: 0 examples [00:00, ? examples/s]

In [23]:
for param in model.parameters():
    param.requires_grad = True


In [26]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 615.06 MiB is free. Process 2240 has 14.14 GiB memory in use. Of the allocated memory 11.49 GiB is allocated by PyTorch, and 2.52 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.save_model()

In [ ]:
from peft import AutoPeftModelForCausalLM


#Load the trained model
model = AutoPeftModelForCausalLM.from_pretrained(
    trainingArgs.output_dir,  # Ensure this is the correct path
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)


tokenizer = AutoTokenizer.from_pretrained(trainingArgs.output_dir)


In [27]:
def prompt_instruction(input_text):
    instruction = "학생의 진로 고민을 토대로 진로 상담사처럼 조언을 해주세요."
    formatted_prompt = instruction + "\n 학생 고민: " + input_text + "\n 상담사 답변:"
    return formatted_prompt


In [28]:
def generate_response_custom(custom_prompt):
    # Format the prompt (assuming prompt_instruction_format is a function you have for formatting)
    prompt = prompt_instruction(custom_prompt)

    # Tokenize the prompt and generate attention mask
    encoding = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    input_ids = encoding.input_ids
    attention_mask = encoding.attention_mask  # Include attention mask

    # Make sure the model and inputs are on the GPU
    local_model = model.to('cuda')
    input_ids = input_ids.to('cuda')
    attention_mask = attention_mask.to('cuda')

    # Generate the model's response
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=300,
        min_length=50,
        max_length=300,
        do_sample=True,
        top_k=40,
        top_p=0.90,
        temperature=1.0,
        num_return_sequences=1,
    )

    # Decode the generated response
    generated_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Print the results
    print(f"-------------")
    print(f"Generated Response:\n{generated_response}")
    print(f"-------------")

In [29]:
custom_prompt = "현재 5학년 2학기이고, 자유전공학부에서 컴퓨터공학과 경영학을 전공하고 있습니다. 개발직종, 대학원, 데이터분석가, 머신러닝 분야 등 다양하게 진로를 생각하고 있었는데 요즘 들어 머신러닝은 너무 잘하는 사람들이 많고, 개발직도 스트레스 많이 받을 거 같아서 어느 방향으로 나아가야할 지 모르겠습니다.  최근에는 데이터 분석가에 관심이 갔는데, 제가 진짜 데이터를 분석하고 싶은 것인지, 아니면 데이터를 활용할 수 있는 다른 직무(예, 마케팅, 기획, 비즈니스분석,PM)을 원하는 건지 잘 모르겠습니다. 한 진로로 잡았으면, 이걸 쭉 잡고 공부 해야하는데 그러지 못하고 방황하고 있는거 같아요. 아직 저한테 딱 맞는 직무를 못 찾는 것도 있을텐데, 제 실력이 너무 부족하다고 느껴요. 다른 사람들은 어떻게 진로를 찾는지, 어떻게 하면 좋을지 상담 받아보고 싶습니다. "
generate_response_custom(custom_prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=300) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


-------------
Generated Response:
학생의 진로 고민을 토대로 진로 상담사처럼 조언을 해주세요.
 학생 고민: 현재 5학년 2학기이고, 자유전공학부에서 컴퓨터공학과 경영학을 전공하고 있습니다. 개발직종, 대학원, 데이터분석가, 머신러닝 분야 등 다양하게 진로를 생각하고 있었는데 요즘 들어 머신러닝은 너무 잘하는 사람들이 많고, 개발직도 스트레스 많이 받을 거 같아서 어느 방향으로 나아가야할 지 모르겠습니다.  최근에는 데이터 분석가에 관심이 갔는데, 제가 진짜 데이터를 분석하고 싶은 것인지, 아니면 데이터를 활용할 수 있는 다른 직무(예, 마케팅, 기획, 비즈니스분석,PM)을 원하는 건지 잘 모르겠습니다. 한 진로로 잡았으면, 이걸 쭉 잡고 공부 해야하는데 그러지 못하고 방황하고 있는거 같아요. 아직 저한테 딱 맞는 직무를 못 찾는 것도 있을텐데, 제 실력이 너무 부족하다고 느껴요. 다른 사람들은 어떻게 진로를 찾는지, 어떻게 하면 좋을지 상담 받아보고 싶습니다. 
 상담사 답변: 현재 진로 고민이 많으시네요~ 머신러닝, 인공지능 같은 개발직종을 하시려면, 프로그래밍 언어를 공부해 놓는 것이 필요합니다. 현재 고2이기 때문에, 고3이 될때 까지 많은 노력을 필요로 합니다. 또한, 데이터를 활용할 수 있는 다른 직무(예, 마케팅, 비즈니스분석,PM 등)를 희망하신다면, 빅데이터를 활용한 데이터분석 직무로 전향하실 수 있습니다. 이는, 데이터를 다룰 수 있는 기초 능력이 필요하고, 빅데이터에 관심을 가지고, 데이터 분석과 관련된 분야에 대해 많은 경험과 지식을 쌓으셔야합니다. 현재 데이터 관련 직무를 하기 위해서는, 전공하시면서, 빅데이터와 관련된 IT 기술을 전공하시는 것을 추천합니다. 또한, 프로그래밍 언어를 공부하시는 것이 좋습니다. 프로그래밍 언어는, 파이썬, C언어, JAVA, JSP, SPRING, APP 등을 배우시는 것을 추천합니다. 고3이 되면, 입시 준비로 시간이 부족하므로, 프로그래밍 언어를 배우시면서, 대학입시와 관련

In [30]:
data= "친화력이 좋은 것이 본인의 장점이며 친구들 사이에서 분위기를 좋게 만들어 주는 역할을 많이 한다. 반면 본인이 변하고 싶어 하는 부분은 계획은 하는데 실천하지 않는 부분이다. 하루 공부량을 정해놓고 하루 공부량을 못 지킨다던가 운동 시간을 미루는 것들이 있다. 이러한 부분을 바꾸려고 많이 노력 한다고 한다. 무언가를 시작하기 전에 상황이나 환경을 살피는 경향이 있다. 또한 책임감이 강해 완벽해지고 싶어 한다. 대학 진학과 취직 중 결정은 못 했지만, 자격증은 15개 정도 있다고 한다.대학에 입학 원서를 넣기 전까지 준비할 것이 별로 없기에 그동안 취업 준비를 하면서 지원원서를 내보고자 한다. 중견기업 재무팀으로의 취업을 원하고 있다. 원하는 곳에 취업하지 못할 경우, 대학에 진학하고자 한다. 직접 사무실을 운영할 수 있다면 세무사를 하고 싶어 한다. 직업 및 진로와 관련된 여러 심리검사를 해봤으며, 추가로 궁금해하거나 묻고 싶은 것이 없다."
generate_response_custom(data)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=300) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-------------
Generated Response:
학생의 진로 고민을 토대로 진로 상담사처럼 조언을 해주세요.
 학생 고민: 친화력이 좋은 것이 본인의 장점이며 친구들 사이에서 분위기를 좋게 만들어 주는 역할을 많이 한다. 반면 본인이 변하고 싶어 하는 부분은 계획은 하는데 실천하지 않는 부분이다. 하루 공부량을 정해놓고 하루 공부량을 못 지킨다던가 운동 시간을 미루는 것들이 있다. 이러한 부분을 바꾸려고 많이 노력 한다고 한다. 무언가를 시작하기 전에 상황이나 환경을 살피는 경향이 있다. 또한 책임감이 강해 완벽해지고 싶어 한다. 대학 진학과 취직 중 결정은 못 했지만, 자격증은 15개 정도 있다고 한다.대학에 입학 원서를 넣기 전까지 준비할 것이 별로 없기에 그동안 취업 준비를 하면서 지원원서를 내보고자 한다. 중견기업 재무팀으로의 취업을 원하고 있다. 원하는 곳에 취업하지 못할 경우, 대학에 진학하고자 한다. 직접 사무실을 운영할 수 있다면 세무사를 하고 싶어 한다. 직업 및 진로와 관련된 여러 심리검사를 해봤으며, 추가로 궁금해하거나 묻고 싶은 것이 없다.
 상담사 답변: 지금의 고민이나 걱정은 대부분 본인에게 달려있다. 본인이 잘하는 것을 찾고, 그와 관련된 분야를 많이 생각하면서 본인만의 진로 고민을 하기 바란다. 그리고 진로에 대한 고민과 관련된 본인의 노력에 대한 부분도 적어보고, 이를 토대로 본인의 부족한 점과 이를 보완하기 위해 노력한 점에 대해 작성해보면 좋겠다. 이는 본인의 성격과 특성과 관련된 부분, 그리고 자신에게 가장 적합한 진로에 대한 고민과 이에 대한 노력에 대해 기록하는 방식으로 작성해도 좋을 것 같다. 진로 고민과 관련된 본인의 경험과 노력에 대한 기록은 추후에도 계속 작성해나가면 본인이 앞으로의 진로 고민을 하거나, 진로에 대한 고민을 하는데 도움이 될 것이다. (출처 : KRI 한국기록원 공식블로그) 대한민국 최초의 'YBM 커리어 커리어 컨설팅' http://blog.ybmcareer.com/ 공유하기 글 요소

In [33]:
data= " 감수성이 풍부하고 여유로운 성격이다. 또한 완벽하게 하고 싶다. 공개된 곳에서 말을 잘하는 장점이 있다. 단점으로는 할 일을 끝까지 미룬 후에 처리하는 습관이 있다. 좋아하는 것은 음악, 게임, 산책이고 운동을 즐기는 편이다. 전공은 금융경영으로 학과에 관련된 직업 중, 증권사, 회계 사무인 쪽 일을 하고 싶다. 전공을 위해 여러 자격증을 취득했고 필요한 자격증도 준비 중이다. 대학 졸업 후 취업 확률이 높아짐을 생각해서 대학 진학에 관해서 관심이 생겼다. 대학에 관심을 두게 된 것이 최근이라 아직 담임 선생님과 진학 상담을 해 본 적은 없다. 학교 성적은 중위권 정도로 점수가 좀 낮다. 수학, 한국사, 국어, 영어 과목에서 등급이 나오며, 제일 취약한 과목은 수학이다. 기초만 알고 있어서 성적이 잘 나오지 않는 상황이며, 학원이나 인터넷 강의 등을 생각해본 적은 있으나, 자격증 취득이 우선인 학교 특성상 자세히 알아보지는 못했다."
generate_response_custom(data)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=300) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-------------
Generated Response:
학생의 진로 고민을 토대로 진로 상담사처럼 조언을 해주세요.
 학생 고민:  감수성이 풍부하고 여유로운 성격이다. 또한 완벽하게 하고 싶다. 공개된 곳에서 말을 잘하는 장점이 있다. 단점으로는 할 일을 끝까지 미룬 후에 처리하는 습관이 있다. 좋아하는 것은 음악, 게임, 산책이고 운동을 즐기는 편이다. 전공은 금융경영으로 학과에 관련된 직업 중, 증권사, 회계 사무인 쪽 일을 하고 싶다. 전공을 위해 여러 자격증을 취득했고 필요한 자격증도 준비 중이다. 대학 졸업 후 취업 확률이 높아짐을 생각해서 대학 진학에 관해서 관심이 생겼다. 대학에 관심을 두게 된 것이 최근이라 아직 담임 선생님과 진학 상담을 해 본 적은 없다. 학교 성적은 중위권 정도로 점수가 좀 낮다. 수학, 한국사, 국어, 영어 과목에서 등급이 나오며, 제일 취약한 과목은 수학이다. 기초만 알고 있어서 성적이 잘 나오지 않는 상황이며, 학원이나 인터넷 강의 등을 생각해본 적은 있으나, 자격증 취득이 우선인 학교 특성상 자세히 알아보지는 못했다.
 상담사 답변: 학교 성적이 안 좋은 이유로 진학 가능성이 낮아 보이네요. 수학이 약점이라 성적이 잘 안 나오는 경우가 많네요. 성적을 올리기 위해 다른 과목에 집중해야 합니다. 그리고 전공이 금융이라면, 금융 쪽으로 가는 게 더 유리하다고 생각이 듭니다. 금융회사는 일반 기업과는 다르게, 금융과 관련된 일을 하는 곳으로 본인이 관심 있는 분야와 일치해야 합니다. 따라서 금융 쪽으로 전공을 잡아야 할 것입니다. 학교가 좋은 곳이면 성적을 올릴 수 있는 방법이 있습니다. 또한, 지금의 성적으로도 자격증을 따면 충분히 취업할 수 있는 방법도 있을 수 있습니다. 대학 진학이 우선이라면, 대학 진학에 대해 알아볼 필요가 있습니다. 전문학교가 아닌 정규 대학으로 진학을 하는 것도 괜찮다고 생각이 듭니다. 만약 전문학교로 진학하고자 한다면, 본인이 희망하는 분야로 취업을 하기 때문에, 본인이 노력을 더 해야

In [32]:
data = "승부욕이 강한 편이고 분위기를 타는 편이 아니라서 공부하는데 주위 환경의 영향을 받지 않는다. 현재 고등학교 진학에 대해 어느 정도 생각해 두었다. 하고 싶은 일로 디자인계열이나 법조계에 관심이 있다. 이 분야에서 일한다면 재미있게 일을 할 수 있으리라 생각한다. 법조계에서는 검사를, 디자인계열에서는 상품디자이너에 가장 관심이 많다. 진로 심리 검사에서 사회성이 좋고 공간지각력도 높으며 학업성취도 높게 나왔다.진로 심리 결과로 예술성, 공간지각력, 수리 능력이 높으면서 기업형, 탐구형, 예술형이 나왔다. 추천직업으로 시각디자인 산업디자인 등이 있다. 해보고 싶은 일인 상품디자인과 잘 맞는다. 상품디자인 다음으로는 의상디자인도 해보고 싶다. 요즘 최대 관심사로는 어떻게 하면 방학을 보람차게 보낼까 하는 생각이다. 방학이라 책을 더 많이 읽으려 하고 계획표를 세워보기로 했다."
generate_response_custom(data)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=300) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-------------
Generated Response:
학생의 진로 고민을 토대로 진로 상담사처럼 조언을 해주세요.
 학생 고민: 승부욕이 강한 편이고 분위기를 타는 편이 아니라서 공부하는데 주위 환경의 영향을 받지 않는다. 현재 고등학교 진학에 대해 어느 정도 생각해 두었다. 하고 싶은 일로 디자인계열이나 법조계에 관심이 있다. 이 분야에서 일한다면 재미있게 일을 할 수 있으리라 생각한다. 법조계에서는 검사를, 디자인계열에서는 상품디자이너에 가장 관심이 많다. 진로 심리 검사에서 사회성이 좋고 공간지각력도 높으며 학업성취도 높게 나왔다.진로 심리 결과로 예술성, 공간지각력, 수리 능력이 높으면서 기업형, 탐구형, 예술형이 나왔다. 추천직업으로 시각디자인 산업디자인 등이 있다. 해보고 싶은 일인 상품디자인과 잘 맞는다. 상품디자인 다음으로는 의상디자인도 해보고 싶다. 요즘 최대 관심사로는 어떻게 하면 방학을 보람차게 보낼까 하는 생각이다. 방학이라 책을 더 많이 읽으려 하고 계획표를 세워보기로 했다.
 상담사 답변: 해보고 싶은 일이 있는데, 지금은 그 쪽을 더 잘하고 싶다는 생각이 커서 그것을 우선순위에 두고 있고, 지금도 하고 싶은 일이 있지만 지금 하고 있는 공부를 잘할 자신이 없다고 합니다. 지금은 성적이나 대학보다는 대학이 아닌 대학 졸업장이 목표인 것 같아요. 앞으로 하고 싶은 일을 위해서는 지금부터 공부를 더 열심히 해야 한다는 생각을 갖고 있군요. 그리고 고등학교 진학은 좀더 생각해보고 결정하겠다는 말을 하고, 대학 진학을 위해 지금부터 열심히 공부하겠다고 합니다. 중학교 공부와 달리, 고등학교 공부는 더 많은 시간과 노력이 필요하다는 것을 알고 있습니다. 해보고 싶은 일이 있다는 말에는 공감하지만 그것만을 위해서 공부하는 것은 의미가 없을 것 같네요. 공부를 더 열심히 해야겠다고 했는데, 그것은 좀더 깊게 공부를 해서, 더 전문적인 분야로 나가고 싶다는 생각인 것 같습니다. 자신이 무엇을 좋아하는지를 찾고, 그것에 맞는 공부를 해나가면 됩